### Univariate LSTM Models

In [1]:
import numpy as np
from numpy import array
from numpy import hstack


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


#### 1. Vanila LSTM

In [4]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)
print(y.shape)

(6, 3, 1)
(6,)


In [5]:
# LSTM model
model = Sequential()
model.add(layer=LSTM(
    units=50, activation='relu', input_shape=(n_steps, n_features))
)
model.add(layer=Dense(units=1))
model.compile(optimizer='adam', loss='mse')

In [6]:
# fit model
model.fit(x=X, y=y, epochs=200, verbose=False)

In [7]:
# demo predictions
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[70]
  [80]
  [90]]]
(1, 3, 1)
1/1 [==============================] - 0s 127ms/sample
[[101.592705]]
(1, 1)


#### 2. Stacked LSTM

In [8]:
# Stacked model
n_features = 1
model = Sequential()
model.add(layer=LSTM(
    units=50, activation='relu', return_sequences=True,
    input_shape=(n_steps, n_features))
)
model.add(layer=LSTM(units=50, activation='relu'))
model.add(layer=Dense(units=n_features))
model.compile(optimizer='adam', loss='mse')

In [9]:
# fit model
model.fit(x=X, y=y, epochs=200, verbose=False)

In [10]:
# demo prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[70]
  [80]
  [90]]]
(1, 3, 1)
1/1 [==============================] - 0s 328ms/sample
[[101.7748]]
(1, 1)


#### 3. Bidirectional LSTM

In [11]:
# Bidirectional model
n_features = 1
model = Sequential()
model.add(layer=Bidirectional(layer=LSTM(
    units=50, activation='relu'),
    input_shape=(n_steps, n_features))
)
model.add(layer=Dense(units=n_features))
model.compile(optimizer='adam', loss='mse')

In [12]:
# fit model
model.fit(x=X, y=y, epochs=200, verbose=False)

In [13]:
# demo prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[70]
  [80]
  [90]]]
(1, 3, 1)
1/1 [==============================] - 0s 254ms/sample
[[101.46179]]
(1, 1)


### Multivariate LSTM Models

In [14]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1)) 
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
print(dataset.shape)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
(9, 3)


In [15]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape)
print(y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2)
(7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


#### 1. Vanilla LSTM

In [16]:
# LSTM model
n_steps = 3
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
model = Sequential()
model.add(layer=LSTM(
    units=50, activation='relu', input_shape=(n_steps, n_features))
)
model.add(layer=Dense(units=1))
model.compile(optimizer='adam', loss='mse')

In [17]:
# fit model
model.fit(x=X, y=y, epochs=200, verbose=False)

In [18]:
# demo prediction
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[ 80  85]
  [ 90  95]
  [100 105]]]
(1, 3, 2)
1/1 [==============================] - 0s 148ms/sample
[[206.0513]]
(1, 1)


### Multi-step LSTM Models
1. Vector Output Model

In [19]:
# multivariate data preparation
from numpy import array
from numpy import hstack
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))                 
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns                
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
print(dataset.shape)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
(9, 3)


In [20]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [21]:
# define model
model = Sequential()
model.add(layer=LSTM(
    units=100, activation='relu', return_sequences=True,
    input_shape=(n_steps_in, n_features))
)
model.add(layer=LSTM(units=100, activation='relu'))
model.add(layer=Dense(units=n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [22]:
model.fit(x=X, y=y, epochs=200, verbose=False)

In [23]:
# demo prediction
x_input = np.array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[70 75]
  [80 85]
  [90 95]]]
(1, 3, 2)
1/1 [==============================] - 0s 242ms/sample
[[186.24826 207.1894 ]]
(1, 2)
